<a href="https://colab.research.google.com/github/leehow-data/Lee_DTSC3020_Fall2025/blob/main/cgh0117_Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [2]:
#1) Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [3]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [4]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    tables = pd.read_html(html, header=0)

    # Pick the first table with at least 3 columns
    df = next((t for t in tables if t.shape[1] >= 3), None)
    if df is None:
        raise ValueError("No table with ≥3 columns found in the given HTML.")

    # Flatten any MultiIndex columns into single level strings
    df.columns = [' '.join(map(str, col)).strip() if isinstance(col, tuple) else str(col).strip()
                  for col in df.columns]

    return df
    raise NotImplementedError("TODO: implement q1_read_table")


def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids."""
    # Standardize column names (trim spaces, title case)
    df.columns = [c.strip() for c in df.columns]

    # Rename common variations just in case (robust to source differences)
    rename_map = {
        'Country': 'Country',
        'Country Name': 'Country',
        'Alpha-2 code': 'Alpha-2',
        'Alpha2': 'Alpha-2',
        'Alpha-3 code': 'Alpha-3',
        'Alpha3': 'Alpha-3',
        'Numeric code': 'Numeric',
        'Numeric': 'Numeric'
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    # Keep only expected columns (ignore extras)
    expected = ['Country', 'Alpha-2', 'Alpha-3', 'Numeric']
    df = df[[c for c in expected if c in df.columns]]

    # Strip whitespace
    for col in ['Country', 'Alpha-2', 'Alpha-3']:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()

    # Uppercase Alpha-2 and Alpha-3
    for col in ['Alpha-2', 'Alpha-3']:
        if col in df.columns:
            df[col] = df[col].str.upper()

    # Convert Numeric to nullable int (e.g., "004" → 4)
    if 'Numeric' in df.columns:
        df['Numeric'] = (
            pd.to_numeric(df['Numeric'], errors='coerce')
            .astype('Int64')  # nullable integer dtype
        )

    # Drop rows missing critical data
    df = df.dropna(subset=['Country', 'Alpha-2', 'Alpha-3'])

    return df
    raise NotImplementedError("TODO: implement q1_clean")


def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    if 'Numeric' not in df.columns:
        raise ValueError("DataFrame must contain a 'Numeric' column.")

    # Sort descending by Numeric, drop rows with missing values in Numeric
    df_sorted = df.dropna(subset=['Numeric']).sort_values(
        by='Numeric', ascending=False
    )

    # Return top-N rows
    return df_sorted.head(top).reset_index(drop=True)
    raise NotImplementedError("TODO: implement q1_sort_top")


In [7]:
# Q1 — Write your answer here
from pathlib import Path

url = "https://www.iban.com/country-codes"
html = requests.get(url).text

# 1. Read
df_raw = q1_read_table(html)

# 2. Clean
df_clean = q1_clean(df_raw)

# 3. Sort and preview top 15
top15 = q1_sort_top(df_clean, top=15)
print(top15)

# 4. Save to CSV
csv_path = Path("data_q1.csv")
df_clean.to_csv(csv_path, index=False)
print(f"\n✅ Saved cleaned data to {csv_path.resolve()}")






                                              Country Alpha-2 Alpha-3  Numeric
0                                              Zambia      ZM     ZMB      894
1                                               Yemen      YE     YEM      887
2                                               Samoa      WS     WSM      882
3                                   Wallis and Futuna      WF     WLF      876
4                  Venezuela (Bolivarian Republic of)      VE     VEN      862
5                                          Uzbekistan      UZ     UZB      860
6                                             Uruguay      UY     URY      858
7                                        Burkina Faso      BF     BFA      854
8                               Virgin Islands (U.S.)      VI     VIR      850
9                      United States of America (the)      US     USA      840
10                       Tanzania, United Republic of      TZ     TZA      834
11                                        Isle of Ma

/tmp/ipython-input-737383951.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, header=0)


# Q1 – Country Codes Extraction and Cleaning

**Source:** [https://www.iban.com/country-codes](https://www.iban.com/country-codes)

### Steps
1. Fetch the HTML page using `requests`.
2. Use `pandas.read_html()` to read all tables and select the first with ≥3 columns.
3. Clean the data:
   - Trim spaces from all text columns.
   - Convert `Alpha-2` and `Alpha-3` to uppercase.
   - Convert `Numeric` codes to integers (nullable, removing leading zeros).
4. Save the cleaned dataset as **`data_q1.csv`**.
5. Sort descending by `Numeric` and print the **Top 15** rows (no charts).

### Output Files
- `data_q1.csv` — Cleaned dataset
- `q1_country_codes.ipynb` — Notebook with all steps
- Console output — Top-15 countries sorted by `Numeric` (descending)

### Limitation
The dataset comes from a public third-party source (IBAN.com) which may not always reflect the most up-to-date ISO 3166-1 country codes. Some territories or special regions may differ from the official ISO list.


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [8]:
import pandas as pd
from bs4 import BeautifulSoup
import re

def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    """
    soup = BeautifulSoup(html, "html.parser")
    items = []

    # Each story row is a .athing element
    for story in soup.select(".athing"):
        rank_tag = story.select_one(".rank")
        title_tag = story.select_one(".titleline a")

        rank = rank_tag.text.strip().replace(".", "") if rank_tag else "0"
        title = title_tag.text.strip() if title_tag else ""
        link = title_tag["href"].strip() if title_tag and title_tag.has_attr("href") else ""

        # The next <tr> with class 'subtext' has details
        subtext = story.find_next_sibling("tr").select_one(".subtext")

        points_tag = subtext.select_one(".score") if subtext else None
        user_tag = subtext.select_one(".hnuser") if subtext else None
        comment_tag = subtext.find_all("a")[-1] if subtext and subtext.find_all("a") else None

        points = points_tag.text if points_tag else "0 points"
        user = user_tag.text if user_tag else ""
        comments_text = comment_tag.text if comment_tag else "0 comments"

        items.append({
            "rank": rank,
            "title": title,
            "link": link,
            "points": points,
            "comments": comments_text,
            "user": user
        })

    return pd.DataFrame(items)


def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    Cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    df = df.copy()

    # Fill missing text fields
    text_cols = ["title", "link", "user"]
    for col in text_cols:
        if col in df.columns:
            df[col] = df[col].fillna("").astype(str).str.strip()

    # Extract digits from numeric-like fields
    for col in ["points", "comments", "rank"]:
        if col in df.columns:
            df[col] = (
                df[col].astype(str)
                .str.extract(r"(\d+)")
                .fillna(0)
                .astype(int)
            )

    return df


def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N."""
    if "points" not in df.columns:
        raise ValueError("DataFrame must contain a 'points' column.")
    df_sorted = df.sort_values("points", ascending=False).reset_index(drop=True)
    return df_sorted.head(top)



In [9]:
# Q2 — Write your answer here

import requests

# Fetch HTML
url = "https://news.ycombinator.com/"
html = requests.get(url).text

# Parse, clean, and sort
df_raw = q2_parse_items(html)
df_clean = q2_clean(df_raw)
top15 = q2_sort_top(df_clean, top=15)

# Save and display
df_clean.to_csv("data_q2.csv", index=False)
print(top15)
print("\n✅ Saved to data_q2.csv")



    rank                                              title  \
0     20                    Why Nextcloud feels slow to use   
1      5             Ask HN: Who is hiring? (November 2025)   
2     14                          The Case Against PGVector   
3     26                        </> Htmx – The Fetch()ening   
4      7  Learning to read Arthur Whitney's C to become ...   
5     15  A visualization of the RGB space covered by na...   
6     17  WebAssembly (WASM) arch support for the Linux ...   
7     16  First recording of a dying human brain shows w...   
8     13  State of Terminal Emulators in 2025: The Erran...   
9      2                                   AI's Dial-Up Era   
10    21                                           VimGraph   
11     9     Ask HN: Who wants to be hired? (November 2025)   
12     8  The Mack Super Pumper was a locomotive engined...   
13    19  Skyfall-GS – Synthesizing Immersive 3D Urban S...   
14    30  Why engineers can't be rational about program